In [2]:
import pandas as pd

# Read the .jsonl file into a DataFrame
df = pd.read_json('prm800k/prm800k/data/phase1_train.jsonl', lines=True)[['question','label']]
df2 = pd.read_json('prm800k/prm800k/data/phase2_train.jsonl', lines=True)[['question','label']]

# Print the DataFrame
dftotal = pd.concat([df, df2], axis=0)

dftotal.reset_index(drop=True, inplace=True)
dftotal

,question,label
0,{'problem': 'How many seconds are in 7.8 minut...,{'steps': [{'completions': [{'text': '7.8 minu...
1,{'problem': 'How many positive two-digit integ...,"{'steps': [{'completions': [{'text': ""Let's ca..."
2,{'problem': 'The fifth and eighth terms of a g...,{'steps': [{'completions': [{'text': 'So we ha...
3,{'problem': 'Find the value of $x$ that satisf...,"{'steps': [{'completions': [{'text': ""Let's fi..."
4,{'problem': 'What is the least three-digit who...,{'steps': [{'completions': [{'text': 'So we wa...
...,...,...
98726,{'problem': 'Find the largest possible value o...,{'steps': [{'completions': [{'text': 'This is ...
98727,"{'problem': 'Alicia has $n$ candies, where $n$...",{'steps': [{'completions': [{'text': 'I need t...
98728,{'problem': 'Find the remainder when $$33818^2...,{'steps': [{'completions': [{'text': 'This pro...
98729,{'problem': 'On the Cartesian plane in which e...,{'steps': [{'completions': [{'text': 'I need t...


In [3]:
dftotal['label'].apply(lambda x: x['finish_reason']).value_counts()

label
found_error    82003
solution       12473
give_up         3980
bad_problem      275
Name: count, dtype: int64

In [ ]:
dftotalfiltered = dftotal[dftotal['label'].apply(lambda x: x['finish_reason'] == 'solution')].reset_index(drop=True)

dftotalfiltered

In [25]:
def extract_chosen_text(label):
    """
    Extracts the text of the chosen completion for each step in the label.
    """
    chosen_texts = []
    for step in label['steps']:
        chosen_index = step['chosen_completion']
        if chosen_index is not None and chosen_index < len(step['completions']):
            chosen_text = step['completions'][chosen_index]['text']
            chosen_texts.append(chosen_text)
    return ' '.join(chosen_texts)

def extract_problem_and_answer(row):
    problem = row['problem']
    answer = row['ground_truth_answer']
    return pd.Series([problem, answer], index=['problem', 'answer'])

dftotalfiltered['steps'] = dftotalfiltered['label'].apply(extract_chosen_text)
# Apply the function to the DataFrame
dftotalfiltered[['problem', 'answer']] = dftotalfiltered['question'].apply(extract_problem_and_answer)
df=dftotalfiltered[['problem', 'steps', 'answer']]
df

,problem,steps,answer
0,How many seconds are in 7.8 minutes?,7.8 minutes is the same as 7 minutes and 0.8 m...,468
1,How many positive two-digit integers leave a r...,So if a number leaves a remainder of 2 when di...,12
2,The fifth and eighth terms of a geometric sequ...,The nth term of a geometric sequence is given ...,315
3,Find the value of $x$ that satisfies $\frac{\s...,Let's first get rid of the square roots in the...,\frac{20}{3}
4,"One day Max says to Liz, ""Out of the 25 people...",I think we should start by letting the number ...,16
...,...,...,...
12468,What is the smallest integer greater than 200 ...,I want to find an integer n that is greater th...,234
12469,"How many numbers are in the list $$ 6,7,10,11,...",I notice that the numbers in the list increase...,47
12470,In isosceles triangle $\triangle ABC$ we have ...,I notice that the problem involves an isoscele...,2\sqrt{2}
12471,"Alicia has $n$ candies, where $n$ is a positiv...",I need to find a three-digit number $n$ that s...,123


In [10]:
df['steps'][1]

"So if a number leaves a remainder of 2 when divided by 8, it's of the form 8n+2. So we want to know the number of positive two-digit integers of the form 8n+2. I think we should just plug in numbers and see what happens. Ok let's start with n=1. 8*1+2=10 which is a two-digit integer. Let's try n=2. 8*2+2=18 which is also a two-digit integer. That would be n=13. Because 8*13+2=106. So the number of positive two-digit integers of the form 8n+2 is 12.\n\n# Answer\n\n12"

In [30]:
train_df = pd.read_csv('ALLtraincompiled.csv')[['Question','Type']]
test_df = pd.read_csv('ALLtestcompiled.csv')[['Question','Type']]
dftotal = pd.concat([train_df, test_df], axis=0)

dftotal.rename(columns={'Question': 'problem'}, inplace=True)
merged_df = pd.merge(df, dftotal, on='problem', how='outer')
merged_df=merged_df.dropna()
merged_df.to_csv('prmclean.csv',index = False)
merged_df

,problem,steps,answer,Type
0,How many seconds are in 7.8 minutes?,7.8 minutes is the same as 7 minutes and 0.8 m...,468,prealgebra
1,How many seconds are in 7.8 minutes?,"I need to convert minutes to seconds, and I kn...",468,prealgebra
2,How many positive two-digit integers leave a r...,So if a number leaves a remainder of 2 when di...,12,number_theory
3,How many positive two-digit integers leave a r...,I want to find all the numbers between 10 and ...,12,number_theory
4,The fifth and eighth terms of a geometric sequ...,The nth term of a geometric sequence is given ...,315,algebra
...,...,...,...,...
12468,What is the slope of the line $2y = -3x + 6$?,So we want to find the slope of the line $2y =...,-\frac32,algebra
12469,Find the slope of the line $3x+5y=20$.,"To find the slope of a line, we need to put it...",-\frac{3}{5},algebra
12470,What is the smallest integer greater than 200 ...,I want to find an integer n that is greater th...,234,prealgebra
12471,"How many numbers are in the list $$ 6,7,10,11,...",I notice that the numbers in the list increase...,47,counting_and_probability


In [28]:
merged_df['steps'][0]

'7.8 minutes is the same as 7 minutes and 0.8 minutes. Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes. And since there are 60 seconds in a minute, then there are 60 * 0.8 = 48 seconds in 0.8 minutes. So, in total, there are 420 + 48 = 468 seconds in 7.8 minutes. Exactly.\n\n# Answer\n\n468'

In [29]:
merged_df['steps'][1]

'I need to convert minutes to seconds, and I know that one minute has 60 seconds. So I can multiply the number of minutes by 60 to get the number of seconds. That means 7.8 minutes times 60 seconds per minute equals 468 seconds. I can check my answer by dividing 468 by 60 and seeing that I get 7.8, which is the original number of minutes. # Answer\n\n468'